<a href="https://colab.research.google.com/github/haironghe03/AutoTomb/blob/main/Helen_25_10_17_Sheet_Music_Cataloging_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature Updates

### New features 2024/12/17: Added fifth column: image name. An image will always appear as a row in the output even if they are skipped.

### New features 2025/10/17: Frontloaded all user settings to the first cell. Program can run with no need to modify anything in any other cells than the first one.

Installation and set up

In [15]:
!pip install openai xlsxwriter pandas

from openai import OpenAI
import os
import csv
import base64
import pandas as pd # Added import for excel functionality

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
from google.colab import drive
drive.mount('/content/drive')

API_KEY = OPENAI_API_KEY

folder_path = "/content/drive/MyDrive/TAWA LOWERRE CONTENTS NOTE & AI PROJECT/AI Project/Run/2011TW-1029/2011TW-1029_2"  # NB: MODIFY BASED ON THE DRIVE PATH IN YOUR COMPUTER WHERE YOU WANT IMAGES READ FROM
folder_name = os.path.basename(folder_path)
csv_file = os.path.join('/content/drive/My Drive', f"output_{folder_name}.csv")  #NB: Modify to where you want csv output to be stored and what name.
xlsx_file = os.path.join('/content/drive/My Drive', f"Output_{folder_name}.xlsx")  #NB: Modify to where you want xlsx output to be stored and what name.

# Standard sheet music columns: 'Title', 'Statement of responsibility', 'Edition', 'City', 'Publisher name', 'Address', 'Plate number', 'Date', 'Image name'
csv_columns = ['theater', 'date', 'title', 'Image name'] #NB: Modify to the column information you need to extract from the images.

#NB: Feel free to store as many prompts below (with unique names); Prompt used in run is selected in cell 2

prompt_text = ("You are an archivist working on creating structured data from an image."
"You will need to manually transcribe the data from the images, and only from the images."
"You will extract the information from the image and present it as a JSON object with four fields: 'theater', 'date', 'title', 'image name'."
"Please don't use markdown to present the result."
"Use ISBD punctuation and RDA (Resource Description & Access) capitalization rules, specific to the German language. For instance, initials and acronyms are to be transcribed without internal spaces, regardless of how they are presented on the image. For example, 'F.K.E. Kennedy', NOT 'F. K. E. Kennedy'. "
"Your response should only contain the data requested. No additional explanation and no extra information. Only transcribe information which is directly stated in the scan. Do not add any outside information, speculation, or generative text. Stick strictly to the text presented in the scan. If you cannot read, or cannot correctly transcribe something, you may return a result of '[illegible]'. "
"Do not use superscript at any time. "
"These are programs from related theaters in Vienna, in 1878. The programs typically present two shows each: a program for the K.K. Hof-Burgtheater on the left panel, and a program for the K.K. Hof-Operntheater on the right panel. You will transcribe the information as requested, beginning with the left-hand panel, ending with the right-hand panel. The fonts vary but are often Fraktur: please transcribe any Fraktur into modern Roman type. Some information presented may be in a Viennese dialect of German: transcribe it as you find it, do not correct it. "
"Ignore any statement of coming attractions. "
"First extract the 'theater' field. The theater name is typically printed above the program panel, at the top of either side of the page. With very few exceptions, the 'K.K. Hof-Burgtheater' will be found at the head of the left panel, and the 'K.K. Hof-Operntheater' will be at the head of the right panel. When there are two panels, extract both theater names in the order they appear, left to right, separated by ' ; '. "
"Second extract the 'date' field, which is often found directly below the theater header on either panel, and often takes this format: 'Dinstag den 1. Jänner 1878'. Extract the date as you find it, in dialect if necessary. DO NOT include the starting time, which would be a phrase beginning with 'Anfang ...' and DO NOT correct any dialect listings into modern German. If this date is the same in both panels, only extract it once. "
"Third extract the title and statement of responsibility into a single field: the 'title' field is typically presented in bold face, and will include a subtitle indicating the number of acts, and a statement of responsibility. Separate the subtitles from the title with this punctuation: ' : ' and separate the statements of responsibility from the subtitles with this punctuation: ' / '. Often you will discover two or more performances listed on the same panel: these additional titles, subtitles, and statements of responsibility should be transcribed in successive order at the end of the 'title' field, separated by this punctuation: ' ; '. When there are two panels, transcribe all 'title' information listed on the left panel as instructed, first, then transcribe all title information listed on the right panel, next, separated by ' ; '. If you encounter an opera or play which is originally in another language with a different standard title, please supply the original title in square brackets after the title but before the subtitle. For instance: 'Don Juan [Don Giovanni]''. If you are unsure as to the original title, simply supply empty square brackets '[]' to indicate that a title has clearly been translated. "
"Capitalization should follow the appropriate usage for the German language, so nouns should always be capitalized, etc. "
"Finally, please format the output as a JSON object with keys: 'theater', 'date', 'title', 'image name' .")

prompt = prompt_text #NB: Select the prompt which you want to use in this run


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.4 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define Function to Get Image Information using the OpenAI API

In [9]:
import requests
import json
import base64
from PIL import Image

def encode_image(path):
    rgb_image = Image.open(path)
    out_path = "output.jpeg"
    rgb_image.save(out_path, format="JPEG")

    with open(out_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def image_to_text(image_path):
    """
    Converts an image to text using OpenAI's API.

    Parameter:
    - image_path: The path to the image file.
    """

    # Function to encode the image to base64
    base64_image = encode_image(image_path)

    # Define the API endpoint
    url = 'https://go.apis.huit.harvard.edu/ais-openai-direct/v1/chat/completions'

    # Set the headers
    headers = {
        'Content-Type': 'application/json',
        'api-key': API_KEY
    }

    # Create the payload
    data = {
        "model": "gpt-4o",
        "messages":     [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
        "max_tokens": 3000
    }

    # print(data)

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"API request failed with status code {response.status_code}: {response.text}")
        return None

    try:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
    except Exception as e:
        print("Failed to decode response:")
        print(response.text)
        raise e

    return content

List Image Files in the Google Drive Folder and Prepare CSV File for Output

In [10]:
# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Sort the image paths by their filenames (natural sort order)
image_files.sort(key=lambda x: os.path.basename(x))

print(image_files)

# Extract the folder name from folder_path
folder_name = os.path.basename(folder_path)

['IMG_2025_10_01_10_23_09S.jpg', 'IMG_2025_10_01_10_23_39S.jpg', 'IMG_2025_10_01_10_23_58S.jpg', 'IMG_2025_10_01_10_24_15S.jpg', 'IMG_2025_10_01_10_24_39S.jpg', 'IMG_2025_10_01_10_24_53S.jpg', 'IMG_2025_10_01_10_25_15S.jpg', 'IMG_2025_10_01_10_26_07S.jpg', 'IMG_2025_10_01_10_28_21S.jpg', 'IMG_2025_10_01_10_28_33S.jpg', 'IMG_2025_10_01_10_28_44S.jpg', 'IMG_2025_10_01_10_29_03S.jpg', 'IMG_2025_10_01_10_29_21S.jpg', 'IMG_2025_10_01_10_29_31S.jpg', 'IMG_2025_10_01_10_30_40S.jpg', 'IMG_2025_10_01_10_30_52S.jpg', 'IMG_2025_10_01_10_31_02S.jpg', 'IMG_2025_10_01_10_31_13S.jpg', 'IMG_2025_10_01_10_31_56S.jpg', 'IMG_2025_10_01_10_32_16S.jpg', 'IMG_2025_10_01_10_32_29S.jpg', 'IMG_2025_10_01_10_32_49S.jpg', 'IMG_2025_10_01_10_33_03S.jpg', 'IMG_2025_10_01_10_33_15S.jpg', 'IMG_2025_10_01_10_34_17S.jpg']


Loop through each image file, extract information using the OpenAI API, and write the data to the CSV file.

In [11]:
# Extract information and write to CSV
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_columns)

    # Create a mapping from column name to index for easier assignment
    col_name_to_index = {name: index for index, name in enumerate(csv_columns)}

    for index, image_file in enumerate(image_files):  # Added enumerate to get the index
        image_path = os.path.join(folder_path, image_file)

        # Initialize row with empty strings based on the number of columns
        row = [""] * len(csv_columns)


        print(f"Processing image {index + 1}: {image_file}")  # Now prints image index and name
        try:
            info = image_to_text(image_path)

            # Debugging line
            print("Debug: Raw API response text", info)

            if info:
                try:
                    data = json.loads(info)

                    # Assign data to the row using the column name to index mapping
                    for key, value in data.items():
                        if key in col_name_to_index:
                            row[col_name_to_index[key]] = value.strip()

                    # Ensure the image file name is always assigned to the correct column
                    if 'Image name' in col_name_to_index:
                        row[col_name_to_index['Image name']] = image_file


                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON for image {image_file}: {e}")
                except KeyError as e:
                    print(f"Missing key in JSON response for image {image_file}: {e}")
        except Exception as e:
            print(f"An error occurred while processing image {image_file}: {e}")


        writer.writerow(row)

print(f'Information extracted and saved to {csv_file}')

# Convert CSV to XLSX
try:
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Use xlsxwriter for conversion
    with pd.ExcelWriter(xlsx_file, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')

    print(f'CSV successfully converted to Excel and saved to {xlsx_file}')

except Exception as e:
    print(f'Error converting CSV to Excel: {e}')

Processing image 1: IMG_2025_10_01_10_23_09S.jpg
Debug: Raw API response text {
"theater": "K.K. Hof-Burgtheater",
"date": "Freitag den 1. Februar 1878",
"title": "Eine Laune : Lustspiel in einem Akt / Alfred de Musset ; Man muß nichts verschwören : Lustspiel in drei Akten nach Alfred de Musset (Il ne faut jurer de rien) für die deutsche Bühne bearbeitet von Lili Kaulser.",
"image name": "theater_program_1878_feb_01.jpg"
}
Processing image 2: IMG_2025_10_01_10_23_39S.jpg
Debug: Raw API response text {"theater":"K.K. Hof-Burgtheater ; K.K. Hof-Operntheater","date":"Samstag den 2. Februar 1878","title":"Eine vornehme Ehe : Schauspiel in fünf Aufzügen und einem Vorspiele / nach Octave Feuillet ; Aida [Aida] : Oper in vier Akten / Text von A. Ghislanzoni ; für die deutsche Bühne bearbeitet von F. Schanz ; Musik von Giuseppe Verdi","image name":"theater_program_2_Februar_1878.png"}
Processing image 3: IMG_2025_10_01_10_23_58S.jpg
Debug: Raw API response text {
"theater": "K.K. Hof-Burgtheat